In [ ]:
!pip install openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet

In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
from google.colab import userdata
from huggingface_hub import login

HF_Token = "your own key"
login(token=HF_Token)

# Dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
!pip install datasets
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 33.0 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset

dpo_dataset = load_dataset("json", data_files="/content/drive/MyDrive/chatbot/dpo/dpo_dataset_V2.json")
print(dpo_dataset)

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 1000
    })
})


In [4]:
dpo_dataset["train"][0]

{'prompt': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 07 Aug 2025\n\nYou are a compassionate medical assistant who responds with empathy and care. Use emojis appropriately 😊💙<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nI have been having alot of catching ,pain and discomfort under my right rib.  If I twist to either side especially my right it feels like my rib actually catches on something and at times I have to stop try to catch my breath and wait for it to subside.  There are times if I am laughing too hard that it will do the same thing but normally its more so if I have twisted or moved  a certain way<|eot_id|>',
 'chosen': "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 07 Aug 2025\n\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nIt sounds like you're experiencing pain when turning to one side, which can be quite uncomfor

# DPO and LoRA

Reference: https://github.com/huggingface/trl/tree/main/examples/research_projects/stack_llama_2/scripts

In [5]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 130.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 101.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
from trl import DPOConfig

DPO_config = DPOConfig(
    output_dir="/content/drive/MyDrive/chatbot/dpo/Llama-3.2-3B",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=1e-6,
    lr_scheduler_type="cosine",
    warmup_steps=50,
    optim="paged_adamw_32bit",
    report_to="tensorboard",
    logging_steps=10,
    save_steps=50,
    bf16=True,
)

In [7]:
from peft import LoraConfig

peft_config = LoraConfig(
        lora_alpha=32,
        lora_dropout=0.05,
        r=16,
        bias="none",
        target_modules=[
          "v_proj",
          "q_proj",
          "k_proj",
          "o_proj"
        ],
        task_type="CAUSAL_LM",
)

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
tokenizer.pad_token = tokenizer.eos_token

sft_model = AutoModelForCausalLM.from_pretrained(
    "/content/drive/MyDrive/chatbot/sft/Llama-3.2-3B/merged_checkpoint",
    low_cpu_mem_usage=True,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
from trl import DPOTrainer

DPO_trainer = DPOTrainer(
    model=sft_model,
    ref_model=None,
    args=DPO_config,
    peft_config=peft_config,
    train_dataset=dpo_dataset["train"],
    processing_class=tokenizer,
)

Extracting prompt in train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [12]:
DPO_trainer.train()

Step,Training Loss
10,0.688300
20,0.691100
30,0.695100
40,0.694200
50,0.684100
60,0.681300
70,0.673500
80,0.654700
90,0.638800
100,0.606100


TrainOutput(global_step=250, training_loss=0.5572338523864746, metrics={'train_runtime': 284.576, 'train_samples_per_second': 3.514, 'train_steps_per_second': 0.879, 'total_flos': 0.0, 'train_loss': 0.5572338523864746, 'epoch': 1.0})

# Merge Model

In [13]:
# find the latest checkpoint
import os

def find_latest_checkpoint(checkpoint_dir):
    checkpoints = [d for d in os.listdir(checkpoint_dir) if d.startswith("checkpoint-")]
    if checkpoints:
        latest = max(checkpoints, key=lambda x: int(x.split("-")[1]))
        return os.path.join(checkpoint_dir, latest)
    return None

from peft import AutoPeftModelForCausalLM

checkpoint_path = find_latest_checkpoint("/content/drive/MyDrive/chatbot/dpo/Llama-3.2-3B")
print(f"Loading checkpoint: {checkpoint_path}")

model = AutoPeftModelForCausalLM.from_pretrained(checkpoint_path, torch_dtype=torch.bfloat16)

Loading checkpoint: /content/drive/MyDrive/chatbot/dpo/Llama-3.2-3B/checkpoint-250


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
model = model.merge_and_unload()

In [15]:
model.save_pretrained("/content/drive/MyDrive/chatbot/dpo/Llama-3.2-3B/merged_checkpoint_v2", safe_serialization=True)

# Test

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
tokenizer.pad_token = tokenizer.eos_token

dpo_model = AutoModelForCausalLM.from_pretrained(
    "./Llama-3.2-3B/merged_checkpoint_v2", #"/content/drive/MyDrive/chatbot/dpo/Llama-3.2-3B/merged_checkpoint_v2",
    low_cpu_mem_usage=True,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    # quantization_config = nf4_config
)

C:\Users\23104\miniconda3\envs\PythonProject\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.19s/it]


In [21]:
import torch
from transformers import pipeline

# Build once and reuse to avoid reloading the model each call
def build_pipe(model, tokenizer):
    return pipeline(
        task="text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.bfloat16,   # change to float16 if needed
        device_map="auto",
    )

def chat_generate(pipe, tokenizer, user_text, system_text=None):
    # 1) Prepare chat messages (system first, then user)
    messages = []
    if system_text:
        messages.append({"role": "system", "content": system_text})
    messages.append({"role": "user", "content": user_text})

    # 2) Render messages with the model's chat template
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True  # adds assistant header to start generation
    )

    # 3) Use both EOS and EOT if available for natural stop
    try:
        eot_id = tokenizer.convert_tokens_to_ids("<|eot_id|>")
        if eot_id is not None and eot_id != tokenizer.unk_token_id:
            eos_ids = [tokenizer.eos_token_id, eot_id]
        else:
            eos_ids = tokenizer.eos_token_id
    except Exception:
        eos_ids = tokenizer.eos_token_id

    # 4) Generate (return only the new text)
    out = pipe(
        prompt,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.95,
        top_p=0.95,
        repetition_penalty=1.03,
        eos_token_id=eos_ids,
        return_full_text=False
    )

    # 5) text-generation pipeline returns plain string
    return out[0]["generated_text"]

In [24]:
pipe = build_pipe(dpo_model, tokenizer)
result = chat_generate(
    pipe, tokenizer,
    user_text="Hello, I am concerned about my diabetes.",
)
print(result)

Device set to use cuda:0


Hi there, I really appreciate your concern about the diabetes in you. I have gone through your query in detail. I understand your concern and would like to help. First of all you should go for dieting. Control your sugar levels by diet. Avoid the high fat, more sugar bearing items. It will help you to control sugar levels. Consult with your doctor and take medication accordingly. This will help you a lot. Take regular check up. Avoid to visit hospital when it is necessary and do not panic.
